# Notebook for exploring and curating the Chinese-English Dictionary

In [202]:
import re

import pandas as pd
import numpy as np
# Load words table
colnames = ['id', 'simplified', 'traditional', 'pinyin', 
            'english', 'grammar', 'concept_cn', 'concept_en', 
            'domain_cn', 'domain_en', 'subdomain_cn', 'subdomain_en',
            'image', 'mp3', 'notes', 'headword']
types = {'id': np.uint32, 'simplified': unicode, 'traditional': unicode, 
        'pinyin': np.object, 'english': np.object, 'grammar': np.object,
        'concept_cn': np.object, 'concept_en': np.object,
        'domain_cn': np.object, 'domain_en': np.object, 'subdomain_cn': np.object,
         'subdomain_en': np.object, 'image': np.object, 'mp3': np.object,
         'notes': np.object, 'headword': np.uint32}
index_col = ['headword', 'id']
df = pd.read_table('../data/dictionary/words.txt', names=colnames, dtype=types,
                  index_col = index_col)
#print df['id'].count() # Number of lexical units, add drop=False to set_index() above

# Unique headwords
#print len(df['headword'].unique())

# Example row
df[df.simplified == '越']
#df.tail()

simplified traditional pinyin      english      grammar  \
headword id                                                              
828      828            越          \N    yuè         more       adverb   
         830            越          \N    yuè      Vietnam  proper noun   
         13695          越          \N    yuè  to get over         verb   
         38269          越          \N    yuè          yue      foreign   

               concept_cn concept_en domain_cn         domain_en subdomain_cn  \
headword id                                                                     
828      828           \N         \N        数量          Quantity           \N   
         830           国家    Country        地理         Geography           亚洲   
         13695         \N         \N        行为           Actions           \N   
         38269         \N         \N        外语  Foreign Language           \N   

               subdomain_en image       mp3  \
headword id                                   
828      828             \N    \N  yue4.mp3   
         830           Asia    \N  yue4.mp3   
         13695           \N    \N  yue4.mp3   
         38269           \N    \N  yue4.mp3   

                                                            notes  
headword id                                                        
828      828    Used in 越A越B constructions to mean as A increa...  
         830                                   Short form for 越南.  
         13695                                                 \N  
         38269                Used to transliterate foreign names

In [203]:
fgs = pd.read_table('../data/dictionary/fgs_mwe.txt', names=colnames, dtype=types)
print fgs['id'].count() # Number of lexical units

print len(fgs['headword'].unique()) # Unique headwords

# Example row
fgs[fgs.simplified == '度一切苦厄']

3307
3236


,id,simplified,traditional,pinyin,english,grammar,concept_cn,concept_en,domain_cn,domain_en,subdomain_cn,subdomain_en,image,mp3,notes,headword
2200,1002201,度一切苦厄,\N,dù yī qiē kŭ è,Overcome All Sufferings,phrase,\N,\N,佛教,Buddhism,佛光山,Fo Guang Shan,\N,\N,Venerable Master Hsing Yun's One-Stroke Callig...,1002201


In [255]:
# Conversions for a word based on a traditional characters
# Input
luid = 45890
tradArr = ["佛說", "法", "滅", "盡", "經"]
sanskrit = u""
english = u"Foshuo Fa Mie Jin Jing"
grammar = "proper noun"
concept = u"\\N\t\\N"
#concept = u"佛\tBuddha"
#subdomain = u"中国佛教\tChinese Buddhism"
subdomain = u"大乘佛教\tMahāyāna Buddhism"
#subdomain = u"\\N\t\\N"

# Generated
domain = u"佛教\tBuddhism"
#domain = u"古文\tClassical Chinese"
traditional = ""
simplified = ""
pinyin = ""
for t in tradArr:
  sDF = df[df.traditional == t]['simplified']
  tDF = df[df.traditional == t]['traditional']
  pDF = df[df.traditional == t]['pinyin']
  if sDF.count() > 0:
    simplified += sDF.iloc[0].decode('utf-8')
    traditional +=tDF.iloc[0].decode('utf-8')
    pinyin += pDF.iloc[0].decode('utf-8') + " "
  else:
    sDF = df[df.simplified == t]['simplified']
    tDF = df[df.simplified == t]['simplified']
    pDF = df[df.simplified == t]['pinyin']
    if sDF.count() > 0:
      simplified += sDF.iloc[0].decode('utf-8')
      traditional += sDF.iloc[0].decode('utf-8')
      pinyin += pDF.iloc[0].decode('utf-8') + " "
    else:
      print "%s not found" % t
pinyin.strip()
print simplified
trad = traditional
if simplified == traditional:
  trad = "\\N"
if sanskrit != "":
  if english != "":
    english = u"%s (%s)" % (sanskrit, english)
  else:
    english = u"%s" % sanskrit
print trad
print pinyin
print english

佛说法灭尽经
佛說法滅盡經
Fó shuō fǎ miè jìn jīng 
Foshuo Fa Mie Jin Jing


In [256]:
# Template for a dictionary entry for a Buddhist term
# Input
# Input
tid = 396
kid = 520

pali = u""
japanese = u""
bl = u"BL '%s'" % sanskrit
bscd = u"BCSD '%s'" % traditional
ccd = u"CCD '%s'" % simplified
fe = u"FE '%s'" % traditional
fgdb = u"FGDB '%s'" % traditional
ghdc = u"GHDC '%s'" % simplified
jebd = u"JEBD '%s'" % japanese
kdc = u"KDC %d" % kid
mw = u"MW '%s'" % sanskrit
t = u"T %d" % tid
refArr = [t, kdc] # Add based on references checked

# Generated
if grammar != "proper noun":
  pinyin = pinyin.replace(" ", "")

# Sanskrit, Pali, and Japanese equivalents
fromLang = ""
if sanskrit != "":
  fromLang = u"From Sanskrit: %s" % sanskrit
if pali != "":
  fromLang += u", Pali: %s" % pali
if japanese != "":
  fromLang += u", Japanese: %s" % japanese
if fromLang != "":
  fromLang += u"; "

# References
ref = u""
for r in refArr:
  if r != "":
    ref += r + "; "
ref = re.sub("; $", "", ref)
if ref != "":
  ref = u"(%s)" % ref

note = u""

notes = u"%s%s%s" % (fromLang, note, ref)
print u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, trad, pinyin, english, grammar, concept, domain, subdomain, notes, luid)

45890	佛说法灭尽经	佛說法滅盡經	Fó shuō fǎ miè jìn jīng 	Foshuo Fa Mie Jin Jing	proper noun	\N	\N	佛教	Buddhism	大乘佛教	Mahāyāna Buddhism	\N	\N	(T 396; KDC 520)	45890


In [257]:
# Template for a dictionary entry for a monastic
concept = u"法师\tMonastic"
daterange = u"490-589; "
notes = u"%sChinese monastic who translated sutras in the Chinese Buddhist canon %s" % (daterange, ref)
print u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, trad, pinyin, english, grammar, concept, domain, subdomain, notes, luid)

45890	佛说法灭尽经	佛說法滅盡經	Fó shuō fǎ miè jìn jīng 	Foshuo Fa Mie Jin Jing	proper noun	法师	Monastic	佛教	Buddhism	大乘佛教	Mahāyāna Buddhism	\N	\N	490-589; Chinese monastic who translated sutras in the Chinese Buddhist canon (T 396; KDC 520)	45890


In [258]:
# Add a title from the Taisho to the dictionary and corpus
# Template for a Taisho title as a word entry in the dictionary

# Input
translator = u"Unknown"
daterange = u"420-479"
genre = u"nirvāṇa"
url = u"http://tripitaka.cbeta.org/T12n0396"
nscrolls = 1
volume = 12
dynasty = u"Liu Song"
datestr = u"2016-09-10"
concept = u"经\tSutra"

# Generated
notes = u"%sthe name of a text in the Chinese Buddhist canon %s" % (
    fromLang, ref)
print u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, trad, pinyin, english, grammar, concept, domain, subdomain, notes, luid)
print

# Full title of the Taisho entry
title = u"%s 《%s》" % (english, traditional)
print u"          <tr>"
print u"            <td>%d</td>" % tid
print u"            <td><a href='/taisho/t0%d.html'>%s</a></td>" % (tid, title)
print u"          </tr>"
print

# Collection entry
entry = u"taisho/t0%d.csv\ttaisho/t0%d.html\t%s\tTranslated by %s\ttaisho/t0%d_00.txt\tTaishō\tSūtra\t%s\t%s" % (
    tid, tid, title, translator, tid, daterange, genre)
print entry
print

# List of scrolls
#print "# Source file, Gloss output file, title"
#for i in range(1, nscrolls + 1):
#  if i < 10:
#    print "taisho/t0%d_0%d.txt\ttaisho/t0%d_0%d.html\tScroll %d" % (tid, i, tid, i, i)
#  else:
#    print "taisho/t0%d_%d.txt\ttaisho/t0%d_%d.html\tScroll %d" % (tid, i, tid, i, i)
#print

scrollsDF = pd.DataFrame({"i": range(1, nscrolls + 1)})
genSource = lambda i: "taisho/t0%d_0%d.txt" % (tid, i)
scrollsDF['source'] = scrollsDF['i'].map(genSource)
genOut = lambda i: "taisho/t0%d_0%d.html" % (tid, i)
scrollsDF['output'] = scrollsDF['i'].map(genOut)
genTitle = lambda i: "Scroll %d" % i
scrollsDF['title'] = scrollsDF['i'].map(genTitle)
filename = "../data/corpus/taisho/t0%d.csv" % tid
cols = ['source', 'output', 'title']
scrollsDF.to_csv(filename, sep = "\t", header = False, columns = cols, index = False)

# Translation of colophon
kReference = u""
if kid != 0:
  if sanskrit != "":
    kReference = u"Sanskrit title and date "
  else:
    kReference = u"Date "
  kReference += u"%s from Lancaster (Lancaster 2004, 'K %d')" % (daterange, kid)
dynastyRef = u""
if dynasty != u"":
  dynastyRef = u"Translated by %s in the %s in %d scroll(s)" % (translator, dynasty, nscrolls)
print u"Volume %d, No. %d" % (volume, tid)
print english
print dynastyRef
print
print "<h4>Notes</h4>"
if kid > 0:
  print
  print kReference
  print
print "English translations: None"
print
print "<h4>Primary Source</h4>"
print u"%s, 《%s》 '%s,' in <i>Taishō shinshū Daizōkyō</i> 《大正新脩大藏經》, in Takakusu Junjiro, ed., (Tokyo: Taishō Shinshū Daizōkyō Kankōkai, 1988), Vol. 12, No. %d, Accessed %s, <a href='%s'>%s</a>." % (
    translator, traditional, english, tid, datestr, url, url)
print
print "<h4>References</h4>"
print """<ol><li>Lancaster, L.R. 2004, <i>The Korean Buddhist Canon: A Descriptive Catalogue</i>, <a href="http://www.acmuller.net/descriptive_catalogue/">http://www.acmuller.net/descriptive_catalogue</a>.</li></ol>"""

45890	佛说法灭尽经	佛說法滅盡經	Fó shuō fǎ miè jìn jīng 	Foshuo Fa Mie Jin Jing	proper noun	经	Sutra	佛教	Buddhism	大乘佛教	Mahāyāna Buddhism	\N	\N	the name of a text in the Chinese Buddhist canon (T 396; KDC 520)	45890

          <tr>
            <td>396</td>
            <td><a href='/taisho/t0396.html'>Foshuo Fa Mie Jin Jing 《佛說法滅盡經》</a></td>
          </tr>

taisho/t0396.csv	taisho/t0396.html	Foshuo Fa Mie Jin Jing 《佛說法滅盡經》	Translated by Unknown	taisho/t0396_00.txt	Taishō	Sūtra	420-479	nirvāṇa

Volume 12, No. 396
Foshuo Fa Mie Jin Jing
Translated by Unknown in the Liu Song in 1 scroll(s)

<h4>Notes</h4>

Date 420-479 from Lancaster (Lancaster 2004, 'K 520')

English translations: None

<h4>Primary Source</h4>
Unknown, 《佛說法滅盡經》 'Foshuo Fa Mie Jin Jing,' in <i>Taishō shinshū Daizōkyō</i> 《大正新脩大藏經》, in Takakusu Junjiro, ed., (Tokyo: Taishō Shinshū Daizōkyō Kankōkai, 1988), Vol. 12, No. 396, Accessed 2016-09-10, <a href='http://tripitaka.cbeta.org/T12n0396'>http://tripitaka.cbeta.org/T12n0396</a>